<a href="https://colab.research.google.com/github/Marcin19721205/BDCaseStudy/blob/main/BDCSZad1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split

In [49]:
path = "sample_data/CharlesBookClub.csv"
df = pd.read_csv(path)

print(df.shape)
print(df.head())

(4000, 24)
   Seq#  ID#  Gender    M   R  F  FirstPurch  ChildBks  YouthBks  CookBks  \
0     1   25       1  297  14  2          22         0         1        1   
1     2   29       0  128   8  2          10         0         0        0   
2     3   46       1  138  22  7          56         2         1        2   
3     4   47       1  228   2  1           2         0         0        0   
4     5   51       1  257  10  1          10         0         0        0   

   ...  ItalCook  ItalAtlas  ItalArt  Florence  Related.Purchase  Mcode  \
0  ...         0          0        0         0                 0      5   
1  ...         0          0        0         0                 0      4   
2  ...         1          0        0         0                 2      4   
3  ...         0          0        0         0                 0      5   
4  ...         0          0        0         0                 0      5   

   Rcode  Fcode  Yes_Florence  No_Florence  
0      4      2             0 

M - Monetary (Total Money spent on books)
R - Recency (Months Since last purchase)
F - Frequency - Total number of purchase


the segmentation process aims to divide customers into homogeneous groups. The homogeneous criterion we need for segmentation is the probability to purchase the offering. In this project,  use 3 variables R, F, M as indicators of this propensity

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Seq#              4000 non-null   int64
 1   ID#               4000 non-null   int64
 2   Gender            4000 non-null   int64
 3   M                 4000 non-null   int64
 4   R                 4000 non-null   int64
 5   F                 4000 non-null   int64
 6   FirstPurch        4000 non-null   int64
 7   ChildBks          4000 non-null   int64
 8   YouthBks          4000 non-null   int64
 9   CookBks           4000 non-null   int64
 10  DoItYBks          4000 non-null   int64
 11  RefBks            4000 non-null   int64
 12  ArtBks            4000 non-null   int64
 13  GeogBks           4000 non-null   int64
 14  ItalCook          4000 non-null   int64
 15  ItalAtlas         4000 non-null   int64
 16  ItalArt           4000 non-null   int64
 17  Florence          4000 non-null  

In [32]:
# podaję min / max zmiennej Florence aby sprawdzić czy bool, oraz sume trafień
fl_min = df["Florence"].min()  # min
fl_max = df["Florence"].max()  # max
fl_sum = df["Florence"].sum()  # suma

fl_min, fl_max, fl_sum


(0, 1, np.int64(338))

In [33]:
#Split 60/40 na train/test - to be used later
train_df, test_df = train_test_split(df, test_size=0.4, random_state=123, shuffle=True)

In [34]:
#sprawdzam czy rzeczywiście Florence jest 0/1 - trochę inna metoda
df["Florence"].dtype, df["Florence"].unique(), df["Florence"].value_counts(dropna=False)

(dtype('int64'),
 array([0, 1]),
 Florence
 0    3662
 1     338
 Name: count, dtype: int64)

# Segmentację od teraz robię na danych treningowych - na poźniej

In [36]:
# czy klient kupił Florence - odsetek jedynek = bazowy response rate
# do użycia w określeniu rr
m = (train_df["Florence"] == 1).mean()
m

np.float64(0.08666666666666667)

# podanie wszystkich segmentów R/F/M z obliczeniem odsetek zakupionych Florence - UWAGA dla Treningowego

In [37]:
#podział na segmenty i sumowanie jaka kategoria ma jaką średnią (ile kupiła książek Florence)
rr = train_df.groupby(["Rcode","Fcode","Mcode"])["Florence"].mean().reset_index(name="rr")
rr


,Rcode,Fcode,Mcode,rr
0,1,1,1,0.000000
1,1,1,2,0.250000
2,1,1,3,0.000000
3,1,1,4,0.266667
4,1,1,5,0.107143
5,1,2,2,0.333333
6,1,2,3,0.000000
7,1,2,4,0.000000
8,1,2,5,0.043478
9,1,3,2,1.000000


In [38]:
# wykres słupkowy, żeby złapać ooziom
import plotly.express as px

rr_plot = rr.copy()  # kopia
rr_plot["RFM"] = rr_plot["Rcode"].astype(str) + rr_plot["Fcode"].astype(str) + rr_plot["Mcode"].astype(str)  # etykieta segmentu
rr_mean = rr_plot["rr"].mean()  # mean(rr)

fig = px.bar(
    rr_plot.sort_values("rr", ascending=False),  # sortuj malejąco
    x="RFM", y="rr",  # osie
    title="RFM względem mean()",  # tytuł
    template="plotly_dark"  # dark
)

fig.add_hline(y=rr_mean, annotation_text=f"mean(rr)={rr_mean:.4f}", annotation_position="top left")  # linia średniej
fig.update_layout(xaxis_title="Segment RFM", yaxis_title="rr", xaxis_tickangle=-90)  # czytelność
fig.show()




dyskretyzacja / kategoryzowanie : z ciągłych (lub liczbowych) R/F/M robie małe, skończone klasy, żeby potem grupować segmenty typu RFM

In [55]:
# jawne przyporządkowanie -> Rcode/Fcode/Mcode

train_df["Rcode"] = np.select(
    [train_df["R"].between(0, 2),
     train_df["R"].between(3, 6),
     train_df["R"].between(7, 12),
     train_df["R"] >= 13],
    [1, 2, 3, 4],
    default=0
).astype(int)

train_df["Fcode"] = np.select(
    [train_df["F"] == 1,
     train_df["F"] == 2,
     train_df["F"] >= 3],
    [1, 2, 3],
    default=0
).astype(int)

train_df["Mcode"] = np.select(
    [train_df["M"].between(0, 25),
     train_df["M"].between(26, 50),
     train_df["M"].between(51, 100),
     train_df["M"].between(101, 200),
     train_df["M"] >= 201],
    [1, 2, 3, 4, 5],
    default=0
).astype(int)



#kombinacje dla segmentu #1

Segment 1 - kombinacje RFM, dla których rr > 2 × (overall response rate) (liczone na zbiorze treningowym).
- wyszła pusta tabela - nie ma kombinacji

In [81]:
#wszystkie kombinacje R/F/M dla segmentu 1 (wg rr > 2*overall_rr)
import numpy as np

overall_rr = train_df["Florence"].mean()  # baseline (train)

rr = (train_df.groupby(["Rcode", "Fcode", "Mcode"])["Florence"]
      .mean()
      .reset_index(name="rr"))

seg1 = (rr.loc[rr["rr"] > 2 * overall_rr, ["Rcode", "Fcode", "Mcode", "rr"]]
        .assign(RFM=lambda d: d["Rcode"].astype(str) + d["Fcode"].astype(str) + d["Mcode"].astype(str))
        .sort_values("rr", ascending=False)
        .reset_index(drop=True))

seg1


,Rcode,Fcode,Mcode,rr,RFM


#kombinacje dla segmentu #2

Segment 2 to kombinacje RFM -  overall response rate < rr ≤ 2 × overall response rate (na train).

In [82]:
# wszystkie kombinacje R/F/M dla segmentu 2 (overall_rr < rr <= 2*overall_rr)
import numpy as np

overall_rr = train_df["Florence"].mean()  # baseline (train)

rr = (train_df.groupby(["Rcode", "Fcode", "Mcode"])["Florence"]
      .mean()
      .reset_index(name="rr"))

seg2 = (rr.loc[(rr["rr"] > overall_rr) & (rr["rr"] <= 2 * overall_rr), ["Rcode", "Fcode", "Mcode", "rr"]]
        .assign(RFM=lambda d: d["Rcode"].astype(str) + d["Fcode"].astype(str) + d["Mcode"].astype(str))
        .sort_values("rr", ascending=False)
        .reset_index(drop=True))

seg2


,Rcode,Fcode,Mcode,rr,RFM
0,1,3,1,0.155340,131
1,2,3,1,0.101928,231
2,1,1,1,0.095890,111
3,1,2,1,0.094937,121


Segment 3 to reszta, czyli rr ≤ overall response rate (pozostałe kombinacje RFM).

In [83]:
# foreword: wszystkie kombinacje R/F/M dla segmentu 3 (rr <= overall_rr)
import numpy as np

overall_rr = train_df["Florence"].mean()  # baseline (train)

rr = (train_df.groupby(["Rcode", "Fcode", "Mcode"])["Florence"]
      .mean()
      .reset_index(name="rr"))

seg3 = (rr.loc[rr["rr"] <= overall_rr, ["Rcode", "Fcode", "Mcode", "rr"]]
        .assign(RFM=lambda d: d["Rcode"].astype(str) + d["Fcode"].astype(str) + d["Mcode"].astype(str))
        .sort_values("rr", ascending=False)
        .reset_index(drop=True))

seg3


,Rcode,Fcode,Mcode,rr,RFM
0,2,1,1,0.063248,211
1,2,2,1,0.062176,221


Nr segmentu (1-3), który jest najbardziej obiecujący (zapewnia największy zysk) z punktu
widzenia sprzedaży książki The Art of History of Florence.

In [86]:
#segment 1-3  wybór najbardziej obiecującego (najwyższy mean(rr))
overall_rr = train_df["Florence"].mean()  # bazowy response rate na train

rr = (train_df.groupby(["Rcode", "Fcode", "Mcode"])["Florence"]
      .mean()
      .reset_index(name="rr"))

rr["segment"] = np.select(  # segmentacja 1-3
    [rr["rr"] > 2 * overall_rr, rr["rr"] > overall_rr],
    [1, 2],
    default=3
).astype(int)

seg_stats = (rr.groupby("segment")["rr"]
             .mean()
             .reset_index(name="mean_rr")
             .sort_values("mean_rr", ascending=False)
             .reset_index(drop=True))

best_segment = int(seg_stats.loc[0, "segment"])  # nr segmentu 1-3
print("Najlepszy segment: ",best_segment)
print("-----------------------")
print(seg_stats)



Najlepszy segment:  2
-----------------------
   segment   mean_rr
0        2  0.112024
1        3  0.062712


In [77]:
# lift chart na test_df; score = rr segmentu (Rcode/Fcode/Mcode); decyle 10%
import numpy as np
import pandas as pd
import plotly.express as px

overall_rr = train_df["Florence"].mean()  # bazowy rr (train) - do liftu

rr = (train_df.groupby(["Rcode", "Fcode", "Mcode"])["Florence"]
      .mean()
      .reset_index(name="rr"))  # score per RFM (train)

test_scored = (test_df.merge(rr, on=["Rcode", "Fcode", "Mcode"], how="left")
               .assign(score=lambda d: d["rr"].fillna(overall_rr)))  # fallback gdy brak segmentu

test_scored = test_scored.sort_values("score", ascending=False).reset_index(drop=True)  # ranking
test_scored["decile"] = (np.floor(np.arange(len(test_scored)) / (len(test_scored) / 10)) + 1).astype(int)  # 1..10
test_scored["decile"] = test_scored["decile"].clip(1, 10)  # safety

lift_tbl = (test_scored.groupby("decile")
            .agg(
                n_customers=("Florence", "size"),            # liczba klientów w decylu
                n_purchases=("Florence", "sum"),             # liczba zakupów (1)
                response_rate=("Florence", "mean"),          # rr w decylu
            )
            .reset_index()
            .assign(
                baseline_rr=overall_rr,                      # baseline (train)
                lift=lambda d: d["response_rate"] / d["baseline_rr"]  # lift = rr_decyl / baseline
            ))

fig = px.bar(
    lift_tbl,
    x="decile",
    y="lift",
    hover_data=["n_customers", "n_purchases", "response_rate", "baseline_rr"],
    title="Lift Chart (TEST) — score=rr(Rcode,Fcode,Mcode) from TRAIN; baseline=overall_rr(TRAIN)",
    template="plotly_dark"
)
fig.update_layout(
    xaxis_title="Decile (1 = najwyższy score, 10 = najniższy)",
    yaxis_title="Lift = response_rate(decile) / baseline_rr(train)"
)
fig.show()

lift_tbl


,decile,n_customers,n_purchases,response_rate,baseline_rr,lift
0,1,160,12,0.07500,0.086667,0.865385
1,2,160,13,0.08125,0.086667,0.937500
2,3,160,12,0.07500,0.086667,0.865385
3,4,160,17,0.10625,0.086667,1.225962
4,5,160,13,0.08125,0.086667,0.937500
5,6,160,14,0.08750,0.086667,1.009615
6,7,160,12,0.07500,0.086667,0.865385
7,8,160,12,0.07500,0.086667,0.865385
8,9,160,11,0.06875,0.086667,0.793269
9,10,160,14,0.08750,0.086667,1.009615
